In [3]:
import streamlit as st
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# -----------------------
# App Config
# -----------------------
st.set_page_config(page_title="Financial Freedom Simulator", layout="wide")

st.title("📈 Financial Freedom Simulator")
st.subheader("Probability-based forecasting with real-life uncertainty")

# -----------------------
# Sidebar Inputs
# -----------------------
st.sidebar.header("Your Inputs")

current_investments = st.sidebar.number_input(
    "Current Investments ($)", 0, 10_000_000, 370_000, step=10_000
)

starting_income = st.sidebar.number_input(
    "Annual Income ($)", 0, 1_000_000, 140_000, step=5_000
)

savings_rate = st.sidebar.slider(
    "Savings Rate (%)", 5, 70, 30
) / 100

annual_expenses = st.sidebar.number_input(
    "Annual Expenses ($)", 0, 500_000, 90_000, step=5_000
)

annual_return_mean = st.sidebar.slider(
    "Expected Market Return (%)", 2.0, 10.0, 6.0
) / 100

annual_return_std = st.sidebar.slider(
    "Market Volatility (%)", 5.0, 25.0, 15.0
) / 100

annual_income_growth = st.sidebar.slider(
    "Annual Income Growth (%)", 0.0, 6.0, 3.0
) / 100

promotion_probability = st.sidebar.slider(
    "Annual Promotion Probability (%)", 0, 30, 12
) / 100

promotion_raise = st.sidebar.slider(
    "Promotion Raise (%)", 5, 40, 20
) / 100

annual_inflation = st.sidebar.slider(
    "Inflation (%)", 1.0, 5.0, 2.5
) / 100

expense_growth_real = st.sidebar.slider(
    "Lifestyle Growth Above Inflation (%)", 0.0, 3.0, 0.5
) / 100

safe_withdrawal_rate = st.sidebar.slider(
    "Safe Withdrawal Rate (%)", 3.0, 5.0, 4.0
) / 100

n_simulations = st.sidebar.slider(
    "Monte Carlo Simulations", 1000, 20000, 5000, step=1000
)

# -----------------------
# Derived Values
# -----------------------
months = 600
monthly_return_mean = (1 + annual_return_mean) ** (1/12) - 1
monthly_return_std = annual_return_std / np.sqrt(12)
monthly_income_growth = (1 + annual_income_growth) ** (1/12) - 1
monthly_inflation = (1 + annual_inflation) ** (1/12) - 1
monthly_expense_growth = (1 + expense_growth_real) ** (1/12) - 1
promotion_probability_monthly = promotion_probability / 12

# -----------------------
# Simulation Function
# -----------------------
def simulate_path():
    balance = current_investments
    income = starting_income
    expenses = annual_expenses

    balances = []
    incomes = []
    expenses_list = []

    for _ in range(months):
        r = np.random.normal(monthly_return_mean, monthly_return_std)

        income *= (1 + monthly_income_growth)

        if np.random.rand() < promotion_probability_monthly:
            income *= (1 + promotion_raise)

        contribution = (income * savings_rate) / 12
        balance = balance * (1 + r) + contribution

        expenses = expenses * (1 + monthly_inflation) * (1 + monthly_expense_growth)

        balances.append(balance)
        incomes.append(income)
        expenses_list.append(expenses)

        if balance * safe_withdrawal_rate >= expenses:
            break

    return balances, incomes, expenses_list

# -----------------------
# Run Simulations
# -----------------------
freedom_months = []
all_balances = []

for _ in range(n_simulations):
    balances, _, expenses = simulate_path()
    all_balances.append(balances)

    for i in range(len(balances)):
        if balances[i] * safe_withdrawal_rate >= expenses[i]:
            freedom_months.append(i)
            break

freedom_years = np.array(freedom_months) / 12

# -----------------------
# Results Summary
# -----------------------
col1, col2, col3 = st.columns(3)

col1.metric("Median Freedom (Years)", round(np.percentile(freedom_years, 50), 1))
col2.metric("75% Confidence (Years)", round(np.percentile(freedom_years, 75), 1))
col3.metric("90% Conservative (Years)", round(np.percentile(freedom_years, 90), 1))

# -----------------------
# Visual 1: Probability Curve
# -----------------------
st.subheader("📊 Probability of Financial Freedom")

sorted_years = np.sort(freedom_years)
cdf = np.arange(len(sorted_years)) / len(sorted_years)

fig, ax = plt.subplots()
ax.plot(sorted_years, cdf)
ax.set_xlabel("Years to Financial Freedom")
ax.set_ylabel("Probability")
ax.grid(True)
st.pyplot(fig)

# -----------------------
# Visual 2: Net Worth Fan Chart
# -----------------------
st.subheader("🌊 Net Worth Uncertainty (Fan Chart)")

max_len = max(len(b) for b in all_balances)
paths = np.array([
    np.pad(b, (0, max_len - len(b)), constant_values=np.nan)
    for b in all_balances
])

percentiles = np.nanpercentile(paths, [10, 25, 50, 75, 90], axis=0)

fig, ax = plt.subplots(figsize=(8, 5))
ax.fill_between(range(max_len), percentiles[0], percentiles[4], alpha=0.2)
ax.fill_between(range(max_len), percentiles[1], percentiles[3], alpha=0.4)
ax.plot(percentiles[2])
ax.set_xlabel("Months")
ax.set_ylabel("Investment Balance")
ax.set_title("Net Worth Fan Chart")
st.pyplot(fig)

# -----------------------
# Visual 3: Income vs Expenses (Sample Path)
# -----------------------
st.subheader("💸 Income vs Expenses (Sample Path)")

sample_balances, sample_incomes, sample_expenses = simulate_path()

fig, ax = plt.subplots()
ax.plot(sample_incomes, label="Income")
ax.plot(sample_expenses, label="Expenses")
ax.legend()
ax.set_xlabel("Months")
ax.set_ylabel("Annual $")
st.pyplot(fig)

# -----------------------
# Footer
# -----------------------
st.caption("This tool shows probabilities, not promises. Use it to guide better decisions.")


2025-12-27 16:28:32.182 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-27 16:28:32.183 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-27 16:28:32.299 
  command:

    streamlit run /Users/kevinclark/opt/anaconda3/lib/python3.8/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-12-27 16:28:32.300 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-27 16:28:32.300 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-27 16:28:32.300 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-27 16:28:32.301 Thread 'MainThread': missing ScriptRunContext! This warning can be 

2025-12-27 16:28:32.338 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-27 16:28:32.338 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-27 16:28:32.339 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-27 16:28:32.339 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-27 16:28:32.342 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-27 16:28:32.343 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-27 16:28:32.343 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-27 16:28:32.344 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator()